# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## For use of COLAB

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  !rm -rf /content/sample_data
  %cd /content/drive/My Drive/projects/ML/learningProcess_courses/DataScience/Introduction/week-4
except:
    %cd Introduction/resources/week-4

[Errno 2] No such file or directory: 'Introduction/resources/week-4'
/Volumes/GoogleDrive/My Drive/projects/ML/learningProcess_courses/DataScience/Introduction/week-4


## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def get_sport_info(file_path, dict):
    sport_df=pd.read_csv(file_path)
    sport_df=sport_df[['team', 'W', 'L']]\
    [sport_df['year']==2018]
    try:
        sport_df=sport_df[sport_df['W'].str.isnumeric()]
    except: pass
    sport_df.replace(to_replace={'\*.*$':'', '\xa0.*$':'', '\+.*$':''}, regex=True, inplace=True)
    sport_df['cities'] = sport_df['team'].apply(lambda x: dict[x])
    sport_df[['W', 'L']] = sport_df[['W', 'L']].astype(int)
    sport_df = sport_df.groupby(by=sport_df['cities']).sum()
    sport_df['ratio'] = sport_df['W'] / (sport_df['W'] + sport_df['L'])
    return sport_df

def get_cities_info():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.replace(to_replace='\[.*\]$', value='', regex=True, inplace=True)
    cities.rename(columns={'Population (2016 est.)[8]':'population_by_region', 'Metropolitan area':'city'}, inplace=True)
    cities = cities[['city', 'population_by_region']]
    cities.set_index('city', inplace=True)
    cities['population_by_region'] = cities['population_by_region'].astype(int)
    return cities

NHL = {'Tampa Bay Lightning':'Tampa Bay Area',
        'Boston Bruins':'Boston',
        'Toronto Maple Leafs':'Toronto',
        'Florida Panthers':'Miami–Fort Lauderdale',
        'Detroit Red Wings':'Detroit',
        'Montreal Canadiens':'Montreal',
        'Ottawa Senators':'Ottawa',
        'Buffalo Sabres':'Buffalo',
        'Washington Capitals':'Washington, D.C.',
        'Pittsburgh Penguins':'Pittsburgh',
        'Philadelphia Flyers':'Philadelphia',
        'Columbus Blue Jackets':'Columbus',
        'New Jersey Devils':'New York City',
        'Carolina Hurricanes':'Raleigh',
        'New York Islanders':'New York City',
        'New York Rangers':'New York City',
        'Nashville Predators':'Nashville',
        'Winnipeg Jets':'Winnipeg',
        'Minnesota Wild':'Minneapolis–Saint Paul',
        'Colorado Avalanche':'Denver',
        'St. Louis Blues':'St. Louis',
        'Dallas Stars':'Dallas–Fort Worth',
        'Chicago Blackhawks':'Chicago',
        'Vegas Golden Knights':'Las Vegas',
        'Anaheim Ducks':'Los Angeles',
        'San Jose Sharks':'San Francisco Bay Area',
        'Los Angeles Kings':'Los Angeles',
        'Calgary Flames':'Calgary',
        'Edmonton Oilers':'Edmonton',
        'Vancouver Canucks':'Vancouver',
        'Arizona Coyotes':'Phoenix'}

NBA = {'New York Knicks':'New York City',
        'Brooklyn Nets':'New York City',
        'Los Angeles Lakers':'Los Angeles',
        'Los Angeles Clippers':'Los Angeles',
        'Golden State Warriors':'San Francisco Bay Area',
        'Chicago Bulls':'Chicago',
        'Dallas Mavericks':'Dallas–Fort Worth',
        'Washington Wizards':'Washington, D.C.',
        'Philadelphia 76ers':'Philadelphia',
        'Boston Celtics':'Boston',
        'Minnesota Timberwolves':'Minneapolis–Saint Paul',
        'Denver Nuggets':'Denver',
        'Miami Heat':'Miami–Fort Lauderdale',
        'Phoenix Suns':'Phoenix',
        'Detroit Pistons':'Detroit',
        'Toronto Raptors':'Toronto',
        'Houston Rockets':'Houston',
        'Atlanta Hawks':'Atlanta',
        'Cleveland Cavaliers':'Cleveland',
        'Charlotte Hornets':'Charlotte',
        'Indiana Pacers':'Indianapolis',
        'Milwaukee Bucks':'Milwaukee',
        'New Orleans Pelicans':'New Orleans',
        'Orlando Magic':'Orlando',
        'Portland Trail Blazers':'Portland',
        'Utah Jazz':'Salt Lake City',
        'San Antonio Spurs':'San Antonio',
        'Sacramento Kings':'Sacramento',
        'Oklahoma City Thunder':'Oklahoma City',
        'Memphis Grizzlies':'Memphis'}

MLB = {'New York Yankees':'New York City',
        'New York Mets':'New York City',
        'Los Angeles Dodgers':'Los Angeles',
        'Los Angeles Angels':'Los Angeles',
        'San Francisco Giants':'San Francisco Bay Area',
        'Oakland Athletics':'San Francisco Bay Area',
        'Chicago Cubs':'Chicago',
        'Chicago White Sox':'Chicago',
        'Texas Rangers':'Dallas–Fort Worth',
        'Washington Nationals':'Washington, D.C.',
        'Philadelphia Phillies':'Philadelphia',
        'Boston Red Sox':'Boston',
        'Minnesota Twins':'Minneapolis–Saint Paul',
        'Colorado Rockies':'Denver',
        'Miami Marlins':'Miami–Fort Lauderdale',
        'Arizona Diamondbacks':'Phoenix',
        'Detroit Tigers':'Detroit',
        'Toronto Blue Jays':'Toronto',
        'Houston Astros':'Houston',
        'Atlanta Braves':'Atlanta',
        'Tampa Bay Rays':'Tampa Bay Area',
        'Pittsburgh Pirates':'Pittsburgh',
        'Cleveland Indians':'Cleveland',
        'Seattle Mariners':'Seattle',
        'Cincinnati Reds':'Cincinnati',
        'Kansas City Royals':'Kansas City',
        'St. Louis Cardinals':'St. Louis',
        'Baltimore Orioles':'Baltimore',
        'Milwaukee Brewers':'Milwaukee',
        'San Diego Padres':'San Diego'}

NFL = {'New York Giants':'New York City',
        'New York Jets':'New York City',
        'Los Angeles Rams':'Los Angeles',
        'Los Angeles Chargers':'Los Angeles',
        'San Francisco 49ers':'San Francisco Bay Area',
        'Oakland Raiders':'San Francisco Bay Area',
        'Chicago Bears':'Chicago',
        'Dallas Cowboys':'Dallas–Fort Worth',
        'Washington Redskins':'Washington, D.C.',
        'Philadelphia Eagles':'Philadelphia',
        'New England Patriots':'Boston',
        'Minnesota Vikings':'Minneapolis–Saint Paul',
        'Denver Broncos':'Denver',
        'Miami Dolphins':'Miami–Fort Lauderdale',
        'Arizona Cardinals':'Phoenix',
        'Detroit Lions':'Detroit',
        'Houston Texans':'Houston',
        'Atlanta Falcons':'Atlanta',
        'Tampa Bay Buccaneers':'Tampa Bay Area',
        'Pittsburgh Steelers':'Pittsburgh',
        'Cleveland Browns':'Cleveland',
        'Seattle Seahawks':'Seattle',
        'Cincinnati Bengals':'Cincinnati',
        'Kansas City Chiefs':'Kansas City',
        'Baltimore Ravens':'Baltimore',
        'Carolina Panthers':'Charlotte',
        'Indianapolis Colts':'Indianapolis',
        'Tennessee Titans':'Nashville',
        'New Orleans Saints':'New Orleans',
        'Buffalo Bills':'Buffalo',
        'Jacksonville Jaguars':'Jacksonville',
        'Green Bay Packers':'Green Bay'}


In [ ]:


def nhl_correlation(): 
    # YOUR CODE HERE
    global NHL
    sport_df = get_sport_info(file_path="assets/nhl.csv", dict=NHL)
    cities = get_cities_info()
    result = sport_df.merge(cities, how='left', left_index=True, right_index=True)
    
    population_by_region = list(result['population_by_region']) # pass in metropolitan area population from cities
    win_loss_by_region = list(result['ratio']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    corr, pvalue = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return corr

nhl_correlation()

0.012308996455744264

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
def nba_correlation():
    # YOUR CODE HERE
    global NBA

    sport_df = get_sport_info(file_path="assets/nba.csv", dict=NBA)
    cities = get_cities_info()
    result = sport_df.merge(cities, how='left', left_index=True, right_index=True)
    
    population_by_region = list(result['population_by_region']) # pass in metropolitan area population from cities
    win_loss_by_region = list(result['ratio']) # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    corr, pvalue = stats.pearsonr(population_by_region, win_loss_by_region)

    return corr

nba_correlation()

-0.17657160252844614

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
def mlb_correlation():
    # YOUR CODE HERE

    global MLB
    sport_df = get_sport_info(file_path="assets/mlb.csv", dict=MLB)
    cities = get_cities_info()
    result = sport_df.merge(cities, how='left', left_index=True, right_index=True)

    population_by_region = list(result['population_by_region']) # pass in metropolitan area population from cities
    win_loss_by_region = list(result['ratio']) # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    corr, pvalue = stats.pearsonr(population_by_region, win_loss_by_region)

    return corr

mlb_correlation()

0.1505230448710485

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
def nfl_correlation():
    # YOUR CODE HERE

    global NFL
    sport_df = get_sport_info(file_path="assets/nfl.csv", dict=NFL)
    cities = get_cities_info()
    result = sport_df.merge(cities, how='left', left_index=True, right_index=True)

    population_by_region = list(result['population_by_region']) # pass in metropolitan area population from cities
    win_loss_by_region = list(result['ratio']) # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    corr, pvalue = stats.pearsonr(population_by_region, win_loss_by_region)

    return corr

nfl_correlation()

0.004922112149349409

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
def sports_team_performance():
    # YOUR CODE HERE
    nhl = sport_df = get_sport_info(file_path="assets/nhl.csv", dict=NHL)['ratio']
    nba = sport_df = get_sport_info(file_path="assets/nba.csv", dict=NBA)['ratio']
    nfl = sport_df = get_sport_info(file_path="assets/nfl.csv", dict=NFL)['ratio']
    mlb = sport_df = get_sport_info(file_path="assets/mlb.csv", dict=MLB)['ratio']

    cities = get_cities_info()

    result = cities.merge(nhl, how='left', left_index=True, right_index=True)\
        .merge(nba, how='left', left_index=True, right_index=True)\
        .merge(nfl, how='left', left_index=True, right_index=True)\
        .merge(mlb, how='left', left_index=True, right_index=True)

    result.drop('population_by_region', axis=1, inplace=True)
    result.columns = ['NHL', 'NBA', 'NFL', 'MLB']

    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

    stats.ttest_rel(result['NFL'], result['NHL'])

    for i in p_values.columns:
        for j in p_values.index:
            if i != j:
                df = result[[i,j]].dropna()
                _, p_values[i][j] = stats.ttest_rel(df[i], df[j])

    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030959,0.802384
NBA,0.941792,NaN,0.022316,0.951046
NHL,0.030959,0.022316,NaN,0.000712
MLB,0.802384,0.951046,0.000712,NaN
